# Grover's Algorithm

Implementation of Grover's algorithm <a name="ref-1"/>[(Grover, 1996)](#cite-Grover) using __[Cirq](https://quantumai.google/cirq)__. The marked state in this example is the 5 qubit state vector $\left| w \right\rangle = \left| 10000 \right\rangle$, and the operators $U_w$ and $U_s$ are implemented as in the paper by Figgatt. et al <a name="ref-2"/>[(Figgatt et al., 2017)](#cite-Figgatt_2017).

In [2]:
import math,random
import numpy as np
import cirq
import statistics
from statistics import mode
from cirq import H, X, Z, CNOT, S,measure

In [10]:
n = 5 #Number of qubits

#Number of iterations required for general n = pi*sqrt(2**n)/4 - 1/2
Nit = math.ceil((np.pi*np.sqrt(2**n)/4 - 0.5))
print('Number of iterations required = %d\n'%(Nit))

qbs = cirq.LineQubit.range(n) #Qubits
qc = cirq.Circuit()

#Intial state |s> = |+++>
#Apply H gate to all qubits
for i in range(n):
    qc.append(H(qbs[i]))
    
print("Circuit:")
print(qc)

Number of iterations required = 4

Circuit:
0: ───H───

1: ───H───

2: ───H───

3: ───H───

4: ───H───


In [11]:
#Oracle
#Uw = I - 2|w><w|
#|w> = |10000>

#Function to add Oracle
def AddOracle(QC):
    for i in range(1,n):
        QC.append(X(qbs[i]))
    
    QC.append(Z(qbs[0]).controlled_by(qbs[1],qbs[2],qbs[3],qbs[4]))
    for i in range(1,n):
        QC.append(X(qbs[i]))


#Amplification: Us = 2|s><s| - I
#|s> = |+++>
#Add amplification as implemented in the paper
#Function to add amplification
def AddAmpl(QC):
    for i in range(n):
        QC.append([H(qbs[i]),X(qbs[i])])
    
    QC.append(Z(qbs[0]).controlled_by(qbs[1],qbs[2],qbs[3],qbs[4]))
    
    for i in range(n):
        qc.append([X(qbs[i]),H(qbs[i])])

In [12]:
#Apply (UsUw) Nit times
for i in range(Nit):
    AddOracle(qc)
    AddAmpl(qc)


print("Circuit:")
print(qc)

Circuit:
0: ───H───────Z───H───X───────Z───X───H───────Z───H───X───────Z───X───H───────Z───H───X───────Z───X───H───────Z───H───X───────Z───X───H───
              │               │               │               │               │               │               │               │
1: ───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───
              │               │               │               │               │               │               │               │
2: ───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───
              │               │               │               │               │               │               │               │
3: ───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───X───@───X───H───
              │               │               │    

In [13]:
#Final state vector before measurement
simulator = cirq.Simulator()
Order_List = [qbs[i] for i in range(n)] #Order list for the results
result = simulator.simulate(qc, qubit_order=Order_List)
print(result)

measurements: (no measurements)
output vector: [-0.00513591+0.j -0.00513581+0.j -0.00513582+0.j -0.00513584+0.j
 -0.00513582+0.j -0.00513584+0.j -0.00513578+0.j -0.00513586+0.j
 -0.00513585+0.j -0.00513581+0.j -0.00513582+0.j -0.00513586+0.j
 -0.00513582+0.j -0.00513586+0.j -0.00513585+0.j -0.00513581+0.j
  0.99959046+0.j -0.00513593+0.j -0.00513577+0.j -0.00513583+0.j
 -0.0051358 +0.j -0.00513587+0.j -0.00513574+0.j -0.0051358 +0.j
 -0.00513588+0.j -0.00513585+0.j -0.00513582+0.j -0.00513582+0.j
 -0.00513582+0.j -0.00513585+0.j -0.00513582+0.j -0.00513582+0.j]


In [14]:
#Add measurement
qc.append(measure(qbs[0],qbs[1],qbs[2],qbs[3],qbs[4],key='Result'))

#print("Circuit:")
#print(qc)

In [15]:
#Simulate the circuit
results = simulator.simulate(qc, qubit_order=Order_List)
print(results)

#Just 3 queries of the oracle will reveal 16 = |10000>
samples=simulator.run(qc,repetitions=100000)
print(samples.histogram(key='Result'))

measurements: Result=10000
output vector: |10000⟩
Counter({16: 99909, 28: 6, 22: 6, 4: 6, 27: 5, 8: 5, 2: 5, 0: 4, 29: 4, 18: 4, 17: 4, 19: 3, 9: 3, 14: 3, 3: 3, 11: 3, 21: 3, 5: 3, 20: 3, 10: 2, 31: 2, 15: 2, 30: 2, 6: 2, 13: 2, 24: 1, 1: 1, 7: 1, 23: 1, 25: 1, 26: 1})


<!--bibtex

@book{MikeIke,
    title = {Quantum Computation and Quantum Information},
    author = {Nielsen, Michael A. and Chuang, Issac L},
    year = {2010},
    publisher = {Cambridge University Press}
}

@article{Grover,
   title={A fast quantum mechanical algorithm for database search},
   url={https://arxiv.org/abs/quant-ph/9605043},
   DOI={10.1145/237814.237866},
   journal={Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (STOC 1996)},
   author={Grover, Love K.},
   year={1996},
   month={July}
}

@article{Figgatt_2017,
   title={Complete 3-Qubit Grover search on a programmable quantum computer},
   volume={8},
   ISSN={2041-1723},
   url={http://dx.doi.org/10.1038/s41467-017-01904-7},
   DOI={10.1038/s41467-017-01904-7},
   number={1},
   journal={Nature Communications},
   publisher={Springer Science and Business Media LLC},
   author={Figgatt, C. and Maslov, D. and Landsman, K. A. and Linke, N. M. and Debnath, S. and Monroe, C.},
   year={2017},
   month={Dec}
}

-->

# References

<a name="cite-Grover"/><sup>[^](#ref-1) </sup>Grover, Love K.. 1996. _A fast quantum mechanical algorithm for database search_. [URL](https://arxiv.org/abs/quant-ph/9605043)

<a name="cite-Figgatt_2017"/><sup>[^](#ref-2) </sup>Figgatt, C. and Maslov, D. and Landsman, K. A. and Linke, N. M. and Debnath, S. and Monroe, C.. 2017. _Complete 3-Qubit Grover search on a programmable quantum computer_. [URL](http://dx.doi.org/10.1038/s41467-017-01904-7)

